# Example 3 - Optimisation routine

In Examples 2a and 2b we computed the Curie depth for a single point using a fixed window. Here, we use `CurieOptimise` object to compute the Curie depth over an entire magnetic anomaly.

`CurieOptimise` inherits all methods from `CurieGrid` in addition to:

- iteratively evaluate the Curie depth across the magnetic anomaly
- add priors to limit spurious Curie depth determinations
- optimise Curie depth parameters using constraints from an objective function

### Contents

- [The inverse problem](#The-inverse-problem)
- [Sensitivity analysis](#Sensitivity analysis)
- [Compare Bouligand to Tanaka](#Compare-Bouligand-to-Tanaka)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# initialise CurieOptimise object
grid = pycurious.CurieOptimise(d, xmin, xmax, ymin, ymax)

In [ ]:
# get centroids

window_size = 200e3
xc_list, yc_list = grid.create_centroid_list(window_size, spacingX=10e3, spacingY=10e3)

print("number of centroids = {}".format(len(xc_list)))

## The inverse problem

It is notoriously difficult to estimate uncertainties from Curie depth determinations. Intuitively, the fewer points in the power spectrum should result in higher uncertainty, but it is difficult to quantify these uncertainties in practise. For [Bouligand *et al.*, 2009](http://doi.wiley.com/10.1029/2009JB006494) it is difficult to determine the values of $\beta$ and $\Delta z$ since both control the slope of the power spectrum at low wavenumbers. Similarly, for [Tanaka *et al.*, 1999](http://linkinghub.elsevier.com/retrieve/pii/S0040195199000724) the lower and upper ranges of the power spectrum used to compute $z_b$ and $z_t$ is highly subjective and can result in significantly different Curie depths.

Here, we aim to assess the uncertainty of Curie depth determinations using a sensitivity analysis. The approach we outline here is not necessarily the most statistically robust methodology, but it is intended as a practical means to compute Curie depth whilst considering its uncertainty.

First we find the best model with no priors to evaluate the heterogeneity in the signal we extract from the magnetic anomaly. We use the commonly used $\ell_2$-norm objective function to calculate misfit. The objective function can be accessed (and modified) under `grid.objective_function`

In [ ]:
# no priors
grid.reset_priors()

beta, zt, dz, C = grid.optimise_routine(window_size, xc_list, yc_list)

# plot results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10,10))

sc1 = ax1.scatter(xc_list, yc_list, c=beta)
sc2 = ax2.scatter(xc_list, yc_list, c=zt)
sc3 = ax3.scatter(xc_list, yc_list, c=dz)
sc4 = ax4.scatter(xc_list, yc_list, c=C)

fig.colorbar(sc1, ax=ax1, label=r"$\beta$")
fig.colorbar(sc2, ax=ax2, label=r"$z_t$")
fig.colorbar(sc3, ax=ax3, label=r"$\Delta z$")
fig.colorbar(sc4, ax=ax4, label=r"$C$")

## Sensitivity analysis

The sensitivity analysis requires defining a prior distribution to sample from. The default is a Gaussian normal distribution,

$$
P(\mathbf{m}) = \frac{1}{\sqrt{2\pi} \sigma_p} \exp \left( \frac{-\mathbf{m}^2}{2 \sigma_p^2} \right)
$$

but other distributions can be defined from the `scipy.stats` module. We repeat the inversion multiple times, sampling different values from $P(\mathbf{m})$ to evaluate the uncertainty in each variable. Note: $\mathbf{m}$ is the variables $\beta, z_t, \Delta z, C$ for the Bouligand approach or $z_t, z_b $ for the Tanaka approach.

Prior distributions are added by

```python
from scipy import stats
beta_p = stats.norm(3.0, 1.0)
grid.add_prior(beta=beta_p)
```

and can be accessed from a dictionary:

```python
prior = grid.prior_pdf['beta'] # stats.norm object
prior = grid.prior['beta'] # stats.norm object arguments (3.0, 1.0)
```

In [ ]:
from scipy import stats

beta_p = stats.norm(3.0, 1.0)
zt_p = stats.norm(0.0, 1.0)
dz_p = stats.norm(12.0, 8.0)
C_p = stats.norm(-17.0, 5.0)

grid.add_prior(beta=beta_p, zt=zt_p, dz=dz_p, C=C_p)

In [ ]:
# number of simulations to run for each centroid
nsim = 10

beta, zt, dz, C = grid.sensitivity_routine(nsim, window_size, xc_list, yc_list)

In [ ]:
fig = plt.figure(figsize=(16,4))


for i, (label, var) in enumerate([('beta', beta), ('zt', zt), ('dz', dz), ('C', C)]):
    
    # prior distribution
    prior = grid.prior_pdf[label]
    p, sigma_p = prior.mean(), prior.std()
    
    # posterior distribution
    P, sigma_P = var[:,0].mean(), var[:,1].mean()
    post = stats.norm(P, sigma_P)
    
    x_samples = np.linspace(p-2*sigma_p, p+2*sigma_p, 100)
    
    # plot pdf
    ax = fig.add_subplot(1,4,i+1, xlabel=label)
    ax.plot(x_samples, prior.pdf(x_samples), label="prior")
    ax.plot(x_samples, post.pdf(x_samples), label="posterior")
    ax.legend()

The precision of the values are properly resolved 